# 假设目前有一个十臂老虎机，每个手臂ai摇动一次有pi概率得到奖励为1，其余情况不得奖，现在求摇动策略使得奖励最高

## 一、建立老虎机模型

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Bandit:
    def __init__(self,k):
        self.probs=np.random.uniform(size=k)
        self.best_arm=np.argmax(self.probs)
        self.max_prob=self.probs[self.best_arm]
        self.k=k
        
    def step(self,a):
        if np.random.rand()<self.probs[a]:
            return 1
        else:
            return 0

np.random.seed(1)
k=10
machine=Bandit(k)
# 获奖最大概率的arm
print(machine.best_arm)
# 最大获奖概率
print(machine.max_prob)

1
0.7203244934421581


## 二、写出老虎机的基础操作，包括累计奖励，累计懊悔

In [3]:
class Solver:
    #自身属性，包括每根拉杆的拉动次数，动作合集，累计懊悔合集
    def __init__(self,bandit):
        self.bandit=bandit
        self.counts=np.zeros(self.bandit.k)
        self.regret=0
        self.action=[]
        self.regrets=[]
    #更新累计懊悔    
    def update_regret(self,a):
        self.regret+=self.bandit.max_prob-self.bandit.probs[a]
        self.regrets.append(self.regret)
     # 返回当次执行的动作，根据选择的算法完善   
    def run_one_step(self):
        raise NotImplementedError
    # 执行num_steps动作
    def run(self,num_steps):
        for _ in range(num_steps):
            a=self.run_one_step()
            self.update_regret(a)
            self.action.append(a)
            self.counts[a]+=1

## 三、贪婪算法

In [4]:
class EpsilonGreedy(Solver):
    def __init__(self,bandit,epsilon=0.01,init_prob=1.0):
        super().__init__(bandit)
        self.epsilon=epsilon
        self.estimates=np.array([init_prob]*self.bandit.k)
        
    def run_one_step(self):
        if np.random.random()<self.epsilon:
            a=np.random.randint(0,self.bandit.k)
        else:
            a=self.bandit.best_arm
        reward=self.bandit.step(a)
        self.estimates[a]+=(reward-self.estimates[a])/(1+self.counts[a])
        return a

In [5]:
np.random.seed(1)
solver=EpsilonGreedy(machine,epsilon=0.01)
solver.run(5000)
print(solver.regret)

20.80852433190333


## 四、上界置信算法